In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
from openpyxl import load_workbook
import time
from datetime import datetime
from datetime import timedelta

from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.inspection import plot_partial_dependence, partial_dependence

import pickle
import sys
import glob
import os
plt.style.use('ggplot')


In [2]:
def add_days(date, days, format_input = '%Y%m%d', format_output = '%Y%m%d'):
    """ Resta o suma dias a una fecha que se le indique en el argumento
    
        :type date: STRING
        :param date: Fecha a la que le queremos adicionar dias

        :type days: INT
        :param days: No. dias que queremos sumar, el numero puede ser negativo para restar dias

        :type format_input: STRING
        :param format_input: Formato en el que introducimos la fecha, fault = '%Y%m%d'

        :type format_output: STRING 
        :param format_input: Formato en el que recibiremos la fecha, fault = '%Y%m%d'

        :raises: TelefonicaError
    """
    final_date = str(datetime.strptime(date,format_input) + timedelta(days = days))[0:10]
    return datetime.strptime(final_date,"%Y-%m-%d").strftime(format_output)



In [3]:
def getIngredioData():
    # En esta parte debo anexar el codigo con el que obtendremos los datos consolidados
    df = pd.read_csv("test_df.csv")
    df.fiscal_year_period = df.fiscal_year_period.astype("str")
    df["month"] = df.fiscal_year_period.map(lambda x: x.split(".")[0].rjust(2, '0'))
    df["year"] = df.fiscal_year_period.map(lambda x: x.split(".")[1].ljust(4, '0'))
    df["date"] = df.year + df.month
    df = df.groupby(["date", "Ship_to_party_code","material"]).sum() \
            .reset_index().rename(columns = {"Ship_to_party_code":"ship_to_party_code"})
    return df




def getVariablesToPredict():
    variables_costos = ["net_corn","raw_material_other", "utilities","waste","repair","labor","ohmfg",
                    "supplies_and_packaging", "supplies_indirect","depreciation","3p_freight_usd","logistics",
                    "cos_other","new_cogs"]
    variables_pred = ["new_cogs", "raw_material_other", "net_corn", "labor", "depreciation", "gross_profit", "new_net_revenue", "n3p_net_revenue"]
    variables_tend = ["{0}_12".format(feat) for feat in variables_pred]
    return variables_costos, variables_pred, variables_tend


In [4]:

def getTargetVariables(df, target):
    df_final = df.sort_values(["ship_to_party_code","material","date"])
    df_final["target_1_month"] = df_final.groupby(["ship_to_party_code","material"])[target].shift(-1)
    df_final["target_2_month"] = df_final.groupby(["ship_to_party_code","material"])[target].shift(-2)
    df_final["target_3_month"] = df_final.groupby(["ship_to_party_code","material"])[target].shift(-3)
    df_final["target_4_month"] = df_final.groupby(["ship_to_party_code","material"])[target].shift(-4)
    df_final["target_5_month"] = df_final.groupby(["ship_to_party_code","material"])[target].shift(-5)
    df_final["target_6_month"] = df_final.groupby(["ship_to_party_code","material"])[target].shift(-6)
    df_final["target_7_month"] = df_final.groupby(["ship_to_party_code","material"])[target].shift(-7)
    df_final["target_8_month"] = df_final.groupby(["ship_to_party_code","material"])[target].shift(-8)
    df_final["target_9_month"] = df_final.groupby(["ship_to_party_code","material"])[target].shift(-9)
    df_final["target_10_month"] = df_final.groupby(["ship_to_party_code","material"])[target].shift(-10)
    df_final["target_11_month"] = df_final.groupby(["ship_to_party_code","material"])[target].shift(-11)
    df_final["target_12_month"] = df_final.groupby(["ship_to_party_code","material"])[target].shift(-12)
    df_final["date"] = pd.to_datetime(df_final.date, format="%Y%m")
    return df_final


In [5]:

def getShiftVariables(df_final):
    df_final["new_cogs_12"] = df_final.groupby(["ship_to_party_code","material"])['new_cogs'].shift(12)
    df_final["raw_material_other_12"] = df_final.groupby(["ship_to_party_code","material"])['raw_material_other'].shift(12)
    df_final["net_corn_12"] = df_final.groupby(["ship_to_party_code","material"])['net_corn'].shift(12)
    df_final["labor_12"] = df_final.groupby(["ship_to_party_code","material"])['labor'].shift(12)
    df_final["depreciation_12"] = df_final.groupby(["ship_to_party_code","material"])['depreciation'].shift(12)
    df_final["gross_profit_12"] = df_final.groupby(["ship_to_party_code","material"])['gross_profit'].shift(12)
    df_final["new_net_revenue_12"] = df_final.groupby(["ship_to_party_code","material"])['new_net_revenue'].shift(12)
    df_final["n3p_net_revenue_12"] = df_final.groupby(["ship_to_party_code","material"])['n3p_net_revenue'].shift(12)
    df_final["month"] = df_final.date.astype("str").map(lambda x: x[5:7])
    return df_final




In [6]:

def fillTendVariables(df_final, variables_tend):
    dict_tend = {}
    for feat in variables_tend:
        dict_tend[feat] = 0
    df_final.fillna(dict_tend, inplace = True)
    return df_final


def createRatioVariables(df_final, variables_pred):
    ratio_variables = ["ratio_{0}".format(feat) for feat in variables_pred]
    for feat in variables_pred:
        df_final["ratio_{0}".format(feat)] = df_final[feat] / (df_final["{0}_12".format(feat)] + 0.001)
    return ratio_variables, df_final



In [7]:
def fillnanValues(df_final):
    target_variables = ['target_1_month', 'target_2_month', 'target_3_month', 'target_4_month',
        'target_5_month', 'target_6_month', 'target_7_month', 'target_8_month',
        'target_9_month', 'target_10_month', 'target_11_month',
        'target_12_month']
    pct_null = df_final.isnull().sum() / df_final.shape[0]
    vars_to_keep = pct_null[pct_null <= 0.3].index
    vars_to_keep = [feat for feat in vars_to_keep if feat.startswith("target_") == False]
    df_final = df_final[vars_to_keep + target_variables]
    f_final = df_final.fillna({"sales_qty_total_mt":0, "n3p_net_revenue":0, "3p_freight_usd":0})
    df_final["3p_sales_qty_total_mt"] = np.where(df_final["3p_sales_qty_total_mt"].isnull(),
                                                df_final.sales_qty_total_mt,
                                                df_final["3p_sales_qty_total_mt"])
    return df_final



In [8]:
def createDummyProduct(df_final):
    list_products = ['030050-000', '020010-102', '026430-000', '06810107CA',
       '026550-000', '06442106CA', '32566109CA', '03401048CA', '06863108CA',
       '011420-000']
    df_final["c_commercial_name"] = np.where(df_final.material.isin(list_products), df_final.material, "other")
    df_final = pd.get_dummies(df_final,prefix=['c_commercial_name'], columns = ['c_commercial_name'], drop_first=True)
    return df_final


def createDummyCustomer(df_final):
    list_customer = ['0000121058', '0000120570', '0000120728', '0000900670']
    df_final["c_ship"] = np.where(df_final.ship_to_party_code.isin(list_customer), df_final.ship_to_party_code, "other")
    df_final = pd.get_dummies(df_final,prefix=['c_ship'], columns = ['c_ship'], drop_first=True)
    df_final = pd.get_dummies(df_final,prefix=['c_month'], columns = ['month'], drop_first=True)
    return df_final




In [9]:
def filterByDate(df_final, today):
    day_max = add_days(today, 365, format_input = '%Y-%m-%d', format_output = '%Y-%m-%d')
    day_max[:7] + "-01"
    df_final = df_final[df_final.date < day_max]
    return df_final


def getDummyVariables(df_final):
    dummy_variables = [feat for feat in df_final.columns if feat.startswith("c_")]
    return dummy_variables



In [10]:
def trainingModel(df_final, variables_costos, dummy_variables, ratio_variables, model_type, target):
    dict_model = {"lasso":Lasso(alpha=0.8), "gbm":GradientBoostingRegressor()}
    if model_type == "xgboost":
        for i in range(1,13):
            X = df_final[df_final["target_{0}_month".format(str(i))].isna() == False][["sales_qty_total_mt","3p_sales_qty_total_mt"] + variables_costos + dummy_variables + ratio_variables]
            y = df_final[df_final["target_{0}_month".format(str(i))].isna() == False]["target_{0}_month".format(str(i))]
            X_train , X_test ,y_train, y_test = train_test_split(X,y, random_state=123, test_size=0.2)
            matrix_train = xgb.DMatrix(X_train, label = y_train)
            matrix_test = xgb.DMatrix(X_test, label = y_test)
            model_xgb = xgb.train(params={'objective':'reg:linear','eval_metric':'rmse'},
                                                  dtrain = matrix_train, num_boost_round = 500, early_stopping_rounds = 20,
                                                  evals = [(matrix_test,'test')])
            print("Train RMSE for {0} month: ".format(str(i)),model_xgb.eval(matrix_train))
            print("Test  RMSE for {0} month: ".format(str(i)),model_xgb.eval(matrix_test))
            print("Train  R2 Score : %.2f"%r2_score(y_train, model_xgb.predict(matrix_train)))
            print("Test R2 Score : %.2f"%r2_score(y_test, model_xgb.predict(matrix_test)))
            path_model = "C:/Users/Alberto Oviedo/OneDrive - The Hackett Group, Inc/Ingredion/Modelos/"
            model_name = "model_{0}_{1}_{2}_month".format(model_type, target, str(i))
            pickle.dump(model_xgb, open(path_model + model_name, 'wb'))
    else:
        for i in range(1,13):
            X = df_final[df_final["target_{0}_month".format(str(i))].isna() == False][["sales_qty_total_mt","3p_sales_qty_total_mt"] + variables_costos + dummy_variables + ratio_variables]
            y = df_final[df_final["target_{0}_month".format(str(i))].isna() == False]["target_{0}_month".format(str(i))]
            X_train , X_test ,y_train, y_test = train_test_split(X,y, random_state=123, test_size=0.2)
            model = dict_model[model_type]
            model.fit(X_train,y_train)
            print("Train RMSE for {0} month: ".format(str(i)), math.sqrt(mean_squared_error(y_train, model.predict(X_train))))
            print("Test  RMSE for {0} month: ".format(str(i)), math.sqrt(mean_squared_error(y_test, model.predict(X_test))))
            print("Train  R2 Score : %.2f"%r2_score(y_train, model.predict(X_train)))
            print("Test R2 Score : %.2f"%r2_score(y_test, model.predict(X_test)))
            path_model = "C:/Users/Alberto Oviedo/OneDrive - The Hackett Group, Inc/Ingredion/Modelos/"
            model_name = "model_{0}_{1}_{2}_month".format(model_type, target, str(i))
            pickle.dump(model, open(path_model + model_name, 'wb'))




In [11]:
df = getIngredioData()
variables_costos, variables_pred, variables_tend = getVariablesToPredict()

In [12]:
today = "2020-01-01"
# target_variables = ["sales_qty_total_mt","3p_sales_qty_total_mt","gross_revenue_usd","discounts_usd","new_net_revenue",
#                    "n3p_net_revenue","net_corn","raw_material_other","utilities","waste","repair","labor","ohmfg",
#                    "supplies_and_packaging", "supplies_indirect","depreciation","3p_freight_usd","logistics","cos_other",
#                    "new_cogs","freight_usd", "gross_profit", "operating_income"]
target_variables = ["utilities","waste","repair","labor","ohmfg",
                   "supplies_and_packaging", "supplies_indirect","depreciation","3p_freight_usd","logistics","cos_other",
                   "freight_usd", "gross_profit", "operating_income"]
models = ["lasso","gbm","xgboost"]
for target in target_variables:
    for model_type in models:
        print("#################################################")
        print("#################################################")
        print("#################################################")
        print("starting with: {0} for model {1}".format(target, model_type))
        print("#################################################")
        print("#################################################")
        print("#################################################")
        df_final = getTargetVariables(df, target)
        df_final = getShiftVariables(df_final)
        df_final = fillTendVariables(df_final, variables_tend)
        ratio_variables, df_final = createRatioVariables(df_final, variables_pred)
        df_final = fillnanValues(df_final)
        df_final = createDummyProduct(df_final)
        df_final = createDummyCustomer(df_final)
        df_final = filterByDate(df_final, today)
        dummy_variables = getDummyVariables(df_final)
        trainingModel(df_final, variables_costos, dummy_variables, ratio_variables, model_type, target)

#################################################
#################################################
#################################################
starting with: utilities for model lasso
#################################################
#################################################
#################################################


C:\Users\Alberto Oviedo\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6484350618628.716, tolerance: 5541872248.7412
  model = cd_fast.enet_coordinate_descent(


Train RMSE for 1 month:  7712.350223745854
Test  RMSE for 1 month:  9379.116095463287
Train  R2 Score : 0.77
Test R2 Score : 0.67


C:\Users\Alberto Oviedo\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7919915147294.227, tolerance: 5757832424.827938
  model = cd_fast.enet_coordinate_descent(


Train RMSE for 2 month:  8839.51161646463
Test  RMSE for 2 month:  7776.689324615172
Train  R2 Score : 0.72
Test R2 Score : 0.72


...

C:\Users\Alberto Oviedo\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 202999998394185.56, tolerance: 50807890098.51958
  model = cd_fast.enet_coordinate_descent(


Train RMSE for 11 month:  56714.15962891401
Test  RMSE for 11 month:  91209.83459289579
Train  R2 Score : 0.20
Test R2 Score : 0.11


C:\Users\Alberto Oviedo\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 263792793742406.8, tolerance: 66310107321.65118
  model = cd_fast.enet_coordinate_descent(


Train RMSE for 12 month:  66257.53500966937
Test  RMSE for 12 month:  63366.30765612172
Train  R2 Score : 0.20
Test R2 Score : 0.10
#################################################
#################################################
#################################################
starting with: operating_income for model gbm
#################################################
#################################################
#################################################
Train RMSE for 1 month:  26953.058970049824
Test  RMSE for 1 month:  78342.93175118825
Train  R2 Score : 0.76
Test R2 Score : 0.00
Train RMSE for 2 month:  27815.043074340876
Test  RMSE for 2 month:  90039.4316185664
Train  R2 Score : 0.78
Test R2 Score : -1.14
Train RMSE for 3 month:  27340.989414134125
Test  RMSE for 3 month:  86818.23801176762
Train  R2 Score : 0.80
Test R2 Score : -0.83
Train RMSE for 4 month:  28245.10154044087
Test  RMSE for 4 month:  81109.5285561302
Train  R2 Score : 0.79
Test R2 Score : -0.6

[9]	test-rmse:59877.84375
[10]	test-rmse:59779.45312
[11]	test-rmse:59762.42578
[12]	test-rmse:59551.87109
[13]	test-rmse:59440.99219
[14]	test-rmse:59431.22656
[15]	test-rmse:59119.14453
[16]	test-rmse:59019.16406
[17]	test-rmse:59033.37891
[18]	test-rmse:59054.90625
[19]	test-rmse:59144.51172
[20]	test-rmse:59053.53516
[21]	test-rmse:59045.76562
[22]	test-rmse:59116.76172
[23]	test-rmse:59147.72266
[24]	test-rmse:59123.58984
[25]	test-rmse:59086.89062
[26]	test-rmse:59001.65625
[27]	test-rmse:58984.91797
[28]	test-rmse:58972.81250
[29]	test-rmse:59138.16406
[30]	test-rmse:59141.71484
[31]	test-rmse:59097.61328
[32]	test-rmse:59087.09375
[33]	test-rmse:59082.95703
[34]	test-rmse:59073.79688
[35]	test-rmse:59068.05859
[36]	test-rmse:59051.33203
[37]	test-rmse:59041.01562
[38]	test-rmse:59055.28906
[39]	test-rmse:59053.47656
[40]	test-rmse:59047.11328
[41]	test-rmse:59028.01953
[42]	test-rmse:59049.46875
[43]	test-rmse:59031.66797
[44]	test-rmse:59027.49609
[45]	test-rmse:59026.28906
[4

## Brazil training

In [13]:
def getIngredioData():
    # En esta parte debo anexar el codigo con el que obtendremos los datos consolidados
    df = pd.read_csv("test_df_Brazil.csv")
    df.fiscal_year_period = df.fiscal_year_period.astype("str")
    df["month"] = df.fiscal_year_period.map(lambda x: x.split(".")[0].rjust(2, '0'))
    df["year"] = df.fiscal_year_period.map(lambda x: x.split(".")[1].ljust(4, '0'))
    df["date"] = df.year + df.month
    df = df.groupby(["date", "Ship_to_party_code","material"]).sum() \
            .reset_index().rename(columns = {"Ship_to_party_code":"ship_to_party_code"})
    return df




def getVariablesToPredict():
    variables_costos = ["net_corn","raw_material_other", "utilities","waste","repair","labor","ohmfg",
                    "supplies_and_packaging", "supplies_indirect","depreciation","3p_freight_usd","logistics",
                    "cos_other","new_cogs"]
    variables_pred = ["new_cogs", "raw_material_other", "net_corn", "labor", "depreciation", "gross_profit", "new_net_revenue", "n3p_net_revenue"]
    variables_tend = ["{0}_12".format(feat) for feat in variables_pred]
    return variables_costos, variables_pred, variables_tend


In [ ]:
def createDummyProduct(df_final):
    list_products = ['03401000CE', '13002000AO', '03005000CE', '10560000CL', '10550000CE']
    df_final["c_commercial_name"] = np.where(df_final.material.isin(list_products), df_final.material, "other")
    df_final = pd.get_dummies(df_final,prefix=['c_commercial_name'], columns = ['c_commercial_name'], drop_first=True)
    return df_final


def createDummyCustomer(df_final):
    list_customer = ['#', '0010023686', '0010010539', '0010010611', '0010024488']
    df_final["c_ship"] = np.where(df_final.ship_to_party_code.isin(list_customer), df_final.ship_to_party_code, "other")
    df_final = pd.get_dummies(df_final,prefix=['c_ship'], columns = ['c_ship'], drop_first=True)
    df_final = pd.get_dummies(df_final,prefix=['c_month'], columns = ['month'], drop_first=True)
    return df_final




In [14]:
def trainingModel(df_final, variables_costos, dummy_variables, ratio_variables, model_type, target):
    dict_model = {"lasso":Lasso(alpha=0.8), "gbm":GradientBoostingRegressor()}
    if model_type == "xgboost":
        for i in range(1,13):
            X = df_final[df_final["target_{0}_month".format(str(i))].isna() == False][["sales_qty_total_mt","3p_sales_qty_total_mt"] + variables_costos + dummy_variables + ratio_variables]
            y = df_final[df_final["target_{0}_month".format(str(i))].isna() == False]["target_{0}_month".format(str(i))]
            X_train , X_test ,y_train, y_test = train_test_split(X,y, random_state=123, test_size=0.2)
            matrix_train = xgb.DMatrix(X_train, label = y_train)
            matrix_test = xgb.DMatrix(X_test, label = y_test)
            model_xgb = xgb.train(params={'objective':'reg:linear','eval_metric':'rmse'},
                                                  dtrain = matrix_train, num_boost_round = 500, early_stopping_rounds = 20,
                                                  evals = [(matrix_test,'test')])
            print("Train RMSE for {0} month: ".format(str(i)),model_xgb.eval(matrix_train))
            print("Test  RMSE for {0} month: ".format(str(i)),model_xgb.eval(matrix_test))
            print("Train  R2 Score : %.2f"%r2_score(y_train, model_xgb.predict(matrix_train)))
            print("Test R2 Score : %.2f"%r2_score(y_test, model_xgb.predict(matrix_test)))
            path_model = "C:/Users/Alberto Oviedo/OneDrive - The Hackett Group, Inc/Ingredion/Modelos/"
            model_name = "model_brazil_{0}_{1}_{2}_month".format(model_type, target, str(i))
            pickle.dump(model_xgb, open(path_model + model_name, 'wb'))
    else:
        for i in range(1,13):
            X = df_final[df_final["target_{0}_month".format(str(i))].isna() == False][["sales_qty_total_mt","3p_sales_qty_total_mt"] + variables_costos + dummy_variables + ratio_variables]
            y = df_final[df_final["target_{0}_month".format(str(i))].isna() == False]["target_{0}_month".format(str(i))]
            X_train , X_test ,y_train, y_test = train_test_split(X,y, random_state=123, test_size=0.2)
            model = dict_model[model_type]
            model.fit(X_train,y_train)
            print("Train RMSE for {0} month: ".format(str(i)), math.sqrt(mean_squared_error(y_train, model.predict(X_train))))
            print("Test  RMSE for {0} month: ".format(str(i)), math.sqrt(mean_squared_error(y_test, model.predict(X_test))))
            print("Train  R2 Score : %.2f"%r2_score(y_train, model.predict(X_train)))
            print("Test R2 Score : %.2f"%r2_score(y_test, model.predict(X_test)))
            path_model = "C:/Users/Alberto Oviedo/OneDrive - The Hackett Group, Inc/Ingredion/Modelos/"
            model_name = "model_brazil_{0}_{1}_{2}_month".format(model_type, target, str(i))
            pickle.dump(model, open(path_model + model_name, 'wb'))




In [15]:
df = getIngredioData()
variables_costos, variables_pred, variables_tend = getVariablesToPredict()

In [16]:
today = "2020-01-01"
target_variables = ["sales_qty_total_mt","3p_sales_qty_total_mt","gross_revenue_usd","discounts_usd","new_net_revenue",
                   "n3p_net_revenue","net_corn","raw_material_other","utilities","waste","repair","labor","ohmfg",
                   "supplies_and_packaging", "supplies_indirect","depreciation","3p_freight_usd","logistics","cos_other",
                   "new_cogs","freight_usd", "gross_profit", "operating_income"]
models = ["lasso","gbm","xgboost"]
for target in target_variables:
    for model_type in models:
        print("#################################################")
        print("#################################################")
        print("#################################################")
        print("starting with: {0} for model {1}".format(target, model_type))
        print("#################################################")
        print("#################################################")
        print("#################################################")
        df_final = getTargetVariables(df, target)
        df_final = getShiftVariables(df_final)
        df_final = fillTendVariables(df_final, variables_tend)
        ratio_variables, df_final = createRatioVariables(df_final, variables_pred)
        df_final = fillnanValues(df_final)
        df_final = createDummyProduct(df_final)
        df_final = createDummyCustomer(df_final)
        df_final = filterByDate(df_final, today)
        dummy_variables = getDummyVariables(df_final)
        trainingModel(df_final, variables_costos, dummy_variables, ratio_variables, model_type, target)

#################################################
#################################################
#################################################
starting with: sales_qty_total_mt for model lasso
#################################################
#################################################
#################################################


C:\Users\Alberto Oviedo\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 682302394.2698818, tolerance: 1980356.7193387505
  model = cd_fast.enet_coordinate_descent(


Train RMSE for 1 month:  130.0956798589969
Test  RMSE for 1 month:  128.22725220582404
Train  R2 Score : 0.93
Test R2 Score : 0.95


C:\Users\Alberto Oviedo\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 999412044.8341056, tolerance: 1935295.7270767908
  model = cd_fast.enet_coordinate_descent(


Train RMSE for 2 month:  162.04551343089773
Test  RMSE for 2 month:  166.14780610623916
Train  R2 Score : 0.90
Test R2 Score : 0.92


...

C:\Users\Alberto Oviedo\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1497918140607588.0, tolerance: 307564196657.0048
  model = cd_fast.enet_coordinate_descent(


Train RMSE for 11 month:  246666.93037295612
Test  RMSE for 11 month:  51380.347732526236
Train  R2 Score : 0.03
Test R2 Score : -1.18


C:\Users\Alberto Oviedo\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1505271298754203.5, tolerance: 307540445331.4366
  model = cd_fast.enet_coordinate_descent(


Train RMSE for 12 month:  253158.66312512403
Test  RMSE for 12 month:  35038.40924758648
Train  R2 Score : 0.02
Test R2 Score : -0.11
#################################################
#################################################
#################################################
starting with: operating_income for model gbm
#################################################
#################################################
#################################################
Train RMSE for 1 month:  140081.22857717655
Test  RMSE for 1 month:  287309.8735950932
Train  R2 Score : 1.00
Test R2 Score : -1.95
Train RMSE for 2 month:  62501.326159139964
Test  RMSE for 2 month:  204393.46571046632
Train  R2 Score : 0.90
Test R2 Score : -47.58
Train RMSE for 3 month:  79200.85300087891
Test  RMSE for 3 month:  215231.1209351448
Train  R2 Score : 0.85
Test R2 Score : -27.02
Train RMSE for 4 month:  79064.23369863952
Test  RMSE for 4 month:  200403.26850352786
Train  R2 Score : 0.83
Test R2 Scor

[1]	test-rmse:110010.03906
[2]	test-rmse:147924.50000
[3]	test-rmse:180975.65625
[4]	test-rmse:209386.10938
[5]	test-rmse:233653.32812
[6]	test-rmse:254373.60938
[7]	test-rmse:272029.75000
[8]	test-rmse:279376.75000
[9]	test-rmse:292105.84375
[10]	test-rmse:297550.15625
[11]	test-rmse:302162.31250
[12]	test-rmse:308495.65625
[13]	test-rmse:311915.59375
[14]	test-rmse:314839.56250
[15]	test-rmse:317353.40625
[16]	test-rmse:319460.50000
[17]	test-rmse:321300.15625
[18]	test-rmse:322871.03125
[19]	test-rmse:324200.03125
Train RMSE for 7 month:  [0]	eval-rmse:65548.765625
Test  RMSE for 7 month:  [0]	eval-rmse:325288.906250
Train  R2 Score : 0.92
Test R2 Score : -54.17
[11:00:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[0]	test-rmse:272718.68750
[1]	test-rmse:276612.75000
[2]	test-rmse:280990.43750
[3]	test-rmse:286872.21875
[4]	test-rmse:294396.18750
[5]	test-rmse: